In [2]:
# element wise square
import torch
tensor = torch.tensor([1,2,3,4,5])
squared_tensor = tensor ** 2
# squared_tensor = torch.square(tensor)
print(squared_tensor)

def time_pytorch_function(func, input):
  start = torch.cuda.Event(enable_timing = True)
  end = torch.cuda.Event(enable_timing = True)

  # warmup
  for _ in range(5):
    func(input)
  start.record()
  func(input)
  end.record()
  torch.cuda.synchronize()
  return start.elapsed_time(end)

b = torch.rand(10000, 10000)

def square_2(squared_tensor):
  return squared_tensor * squared_tensor

def square_3(squared_tensor):
  return squared_tensor ** 2

time_pytorch_function(torch.square, b)
time_pytorch_function(square_2, b)
time_pytorch_function(square_3, b)

print("=============")
print("Profiling torch.square")
print("=============")

# now profile each function using pytorch profiler
with torch.autograd.profiler.profile(use_cuda = True) as prof:
  square_2(b)
print(prof.key_averages().table(sort_by = "cuda_time_total", row_limit = 10))
print("=============")
print("Profiling squared_tensor * squared_tensor")
print("=============")

with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_2(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

print("=============")
print("Profiling squared_tensor ** 2")
print("=============")

with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_3(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

tensor([ 1,  4,  9, 16, 25])
Profiling torch.square


<ipython-input-2-e915040dd287>:38: FutureWarning: The attribute `use_cuda` will be deprecated soon, please use ``use_device = 'cuda'`` instead.
  with torch.autograd.profiler.profile(use_cuda = True) as prof:


-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::mul        99.98%     197.639ms        99.98%     197.639ms     197.639ms     197.705ms       100.00%     197.705ms     197.705ms             1  
          cudaEventRecord         0.01%      22.731us         0.01%      22.731us      11.366us       0.000us         0.00%       0.000us       0.000us             2  
    cudaDeviceSynchronize         0.01%      18.794us         0.01%      18.794us      18.794us       0.000us         0.00%       0.000us       0.000us        

<ipython-input-2-e915040dd287>:45: FutureWarning: The attribute `use_cuda` will be deprecated soon, please use ``use_device = 'cuda'`` instead.
  with torch.autograd.profiler.profile(use_cuda=True) as prof:


-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::mul        99.98%     214.587ms        99.98%     214.587ms     214.587ms     214.656ms       100.00%     214.656ms     214.656ms             1  
          cudaEventRecord         0.01%      32.190us         0.01%      32.190us      16.095us       0.000us         0.00%       0.000us       0.000us             2  
    cudaDeviceSynchronize         0.01%      18.718us         0.01%      18.718us      18.718us       0.000us         0.00%       0.000us       0.000us        

<ipython-input-2-e915040dd287>:54: FutureWarning: The attribute `use_cuda` will be deprecated soon, please use ``use_device = 'cuda'`` instead.
  with torch.autograd.profiler.profile(use_cuda=True) as prof:


-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                aten::pow        99.97%     208.006ms        99.98%     208.021ms     208.021ms     208.067ms        99.99%     208.079ms     208.079ms             1  
                 aten::to         0.00%       1.941us         0.00%       1.941us       1.941us       7.000us         0.00%       7.000us       7.000us             1  
        aten::result_type         0.00%       2.334us         0.00%       2.334us       2.334us       5.000us         0.00%       5.000us       5.000us        